# <font color=blue>#StockNews 2018</color>
#### <font color=green>[by Ethan Shaotran]</color>

## Imports:

In [12]:
#MAINLINE-DL Model:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import model_from_json #Reload a model
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import KFold
#from sklearn.preprocessing import StandardScaler
#from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pylab
import numpy as np
import pandas as pd
from pandas import read_csv
import requests
from contextlib import closing
import math
import csv
import os
from datetime import datetime, timedelta
from random import randint

#Alpha Vantage - Data Point Requests:
apikey = 'VPRQOQ85W9KS1DXH'


#News Articles URL Searches: - news-corpus-builder
from news_corpus_builder import NewsCorpusGenerator
corpus_dir = '.' 
NewsGen = NewsCorpusGenerator(corpus_dir,'flat')

#News Article Text Scraping:
from newspaper import Article #USED LATER

#Numpy Random
seed = 7
np.random.seed(seed)

## Functions:

#### [F] Collect Historical Data Points

In [13]:
def CollectData(Ticker):
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=" + Ticker + "&outputsize=full&apikey=" + apikey + "&datatype=csv"
    
    StockHistory = pd.read_csv(url)
#     StockHistory.drop(StockHistory.columns[1], axis=1, inplace=True) #1
#     StockHistory.drop(StockHistory.columns[1], axis=1, inplace=True) #2
#     StockHistory.drop(StockHistory.columns[1], axis=1, inplace=True) #3
#     StockHistory.drop(StockHistory.columns[1], axis=1, inplace=True) #4
#     StockHistory.drop(StockHistory.columns[2], axis=1, inplace=True) #6
#     StockHistory.drop(StockHistory.columns[2], axis=1, inplace=True) #7
#     StockHistory.drop(StockHistory.columns[2], axis=1, inplace=True) #8
    StockHistory.drop(StockHistory.columns[[1,2,3,4,6,7,8]], axis=1, inplace=True)
    print("Collected data from " + Ticker + " for " + str(StockHistory.shape[0]) + " days!")
    return StockHistory


#### [F] Collect News URLS

In [3]:
def GetNewsURLs(Ticker):
    links = NewsGen.google_news_search(Ticker+' stocks',100)
    URLList=[]
    for i in range(0,len(links)-1):
        URLLink = links[i][1]
        if not (SourceInt(URLLink) == 0):
            URLList.append(URLLink)
    print("Found " + str(len(URLList)) + " Articles for Stock " + Ticker)
    return URLList
        

#### [F] Extract Text from URL

In [8]:
def URLtoText (url):
    
    article = Article(url)
    article.download()
    article.parse()
    articletext = article.text
    articledate = article.publish_date

    return (articletext,articledate)

In [9]:
def URLtoTextSummary (url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    articletext = article.summary
    articledate = article.publish_date

    return (articletext,articledate)

#### [F] Get Source of URL

In [5]:
def SourceInt (url):
    url = url.lower()
    if ('cnn' in url):
        sourceint = 1
    elif ('fool.com' in url):
        sourceint = 2
    elif ('finance.yahoo.com' in url):
        sourceint = 3
    elif ('investopedia.com' in url):
        sourceint = 4
    elif ('businessinsider.com' in url):
        sourceint = 5
    elif ('marketwatch.com' in url):
        sourceint = 6
    elif ('forbes.com' in url):
        sourceint = 7
    elif ('thestreet.com' in url):
        sourceint = 8
    elif ('fortune.com' in url):
        sourceint = 9
    elif ('cnbc.com' in url):
        sourceint = 10
    elif ('nasdaq.com' in url):
        sourceint = 11
    elif ('investorplace.com' in url):
        sourceint = 12
    elif ('seekingalpha.com' in url):
        sourceint = 13
    elif ('nytimes' in url):
        sourceint = 14
    elif ('cnbc' in url):
        sourceint = 15
    elif ('apple' in url):
        sourceint = 16
    elif ('techcrunch' in url):
        sourceint = 17
    elif ('huff' in url):
        sourceint = 18
    elif ('fox' in url):
        sourceint = 19
    elif ('usatoday' in url):
        sourceint = 20
    elif ('reuters' in url):
        sourceint = 21
    elif ('npr' in url):
        sourceint = 22
    elif ('nbc' in url):
        sourceint = 23
    elif ('fox' in url):
        sourceint = 24
    elif ('wsj' in url):
        sourceint = 25
    elif ('marketwatch' in url):
        sourceint = 26
    elif ('bloomberg' in url):
        sourceint = 27
    elif ('guardian' in url):
        sourceint = 28
    elif ('usnews' in url):
        sourceint = 29
    elif ('livetradingnews' in url):
        sourceint = 30
    elif ('streetinsider' in url):
        sourceint = 31
    elif ('stocknewsjournal' in url):
        sourceint = 32
    else: sourceint = 0
    return sourceint

#### [F] Calculate Slope of Stock Data Points

In [14]:
def getSlope (articledate, look_ahead, backorahead):
    articledate = articledate.strftime('%Y-%m-%d')
    t = SearchCSVForDate(articledate)
    
    #Calculate X avg, Create list of X Values
    Xavg = 0
    Xlist = []
    if backorahead == 1:
        for x in range(t+1,t-look_ahead,-1):
            Xavg += x
            Xlist.append(x)
    else:
        for x in range(t+look_ahead,t,-1):
            Xavg += x
            Xlist.append(x)
    Xavg /= len(Xlist)
    Xlist.reverse()
        
    #Calculate Y avg, Create list of Y Values
    Yavg = 0
    Ylist = []
    if backorahead == 1:
        for x in range(t+1,t-look_ahead,-1):
            y = StockHistory.get_value(x,"adjusted_close")
            Yavg += y
            Ylist.append(y)
    else:
        for x in range(t+look_ahead,t,-1):
            y = StockHistory.get_value(x,"adjusted_close")
            Yavg += y
            Ylist.append(y)
    Yavg /= len(Ylist)
    
    TopCalculation = 0;
    for i in (0,len(Xlist)-1): #Top Part
        TopCalculation += (Xlist[i] - Xavg)*(Ylist[i] - Yavg)
        
    BottomCalculation = 0;
    for i in (0,len(Xlist)-1): #Bottom Part
        BottomCalculation += math.pow((Xlist[i] - Xavg), 2)
        
    Slope = TopCalculation/BottomCalculation
        
    return Slope;

#### [F] Get CSV Row # For Date

In [15]:
def SearchCSVForDate (articledate):
    return StockHistory[StockHistory['timestamp'] == articledate].index.tolist()[0]

#### [F] Calculate Textual Sentiment

In [16]:
def getSentiment (ListedText):
    posList = []
    negList = []
    
    #Calculate Positivity:
    Positivity = 0
    PositiveWords = open('MasterDictionaryFiles/PositiveWords.txt', 'r').read()
    for i in range(0,len(ListedText)):
        if (("_"+ListedText[i].upper() + "_") in PositiveWords and not (ListedText[i] in posList)):
            #print("+ " + ListedText[i])
            posList.append(ListedText[i])
            Positivity += 1
         
    #Calculate Negativity:
    Negativity = 0
    NegativeWords = open('MasterDictionaryFiles/NegativeWords.txt', 'r').read()
    for i in range(0,len(ListedText)):
        if (("_"+ListedText[i].upper() + "_") in NegativeWords and not (ListedText[i] in negList)):
            #print("- " + ListedText[i])
            negList.append(ListedText[i])
            Negativity += 1
    
    return Positivity, Negativity

## Training:

#### Define URLs for Training Dataset

In [199]:
URLListAAPL = [
'http://money.cnn.com/2007/01/09/technology/apple_jobs/',
'https://seekingalpha.com/article/50765-apples-iphone-builds-huge-lead-in-customer-satisfaction',
'http://www.cnn.com/2008/TECH/11/18/iphone.game.developer/',
'https://www.forbes.com/2009/01/26/steve-jobs-health-tech-enter-cz_rb_0126stevejobs.html',
'http://www.cnn.com/2009/TECH/06/29/steve.jobs.health.work/index.html',
'http://money.cnn.com/2010/01/27/technology/ipad_apple_stock/',
'http://www.cnn.com/2010/TECH/ptech/04/03/apple.ipad.sale/index.html',
'http://www.businessinsider.com/apples-1-million-ipads-in-context-2010-5',
'http://www.marketwatch.com/story/apple-gets-boost-from-ipad-shortage-2010-04-14',
'https://www.forbes.com/sites/heatherstruck/2011/01/17/steve-jobs-sick-leave-could-slow-down-apple-stock/#192f91ce3c02',
'https://www.cnbc.com/id/41118102',
'http://money.cnn.com/2011/01/18/technology/apple_jobs/index.htm',
'http://www.businessinsider.com/steve-jobs-leave-effect-product-2011-1',
'http://money.cnn.com/2011/07/19/technology/apple_q3_earnings/index.htm',
'http://www.marketwatch.com/story/apple-expected-to-set-up-strong-second-half-2011-07-15',
'http://www.marketwatch.com/story/apple-unveils-new-operating-system-for-mac-2011-06-06',
'https://www.thestreet.com/story/11145528/1/icloud-impact-on-apple-stock.html',
'https://investorplace.com/2011/03/tablet-wars-apple-ipad-vs-the-competition/#.WbWoYsbMz-Y',
'https://seekingalpha.com/article/265146-the-rise-and-rise-of-apple-time-for-a-split',
'https://seekingalpha.com/article/262746-apple-investors-time-for-caution-heading-into-q2-earnings',
'http://www.cnn.com/2011/TECH/innovation/07/29/apple.cash.government/index.html',
'https://www.forbes.com/sites/scottdecarlo/2011/08/11/the-worlds-25-most-valuable-companies-apple-is-now-on-top/#12ff0d993814',
'http://www.marketwatch.com/story/apple-crowned-no-1-with-biggest-market-cap-2011-08-10',
'https://www.forbes.com/sites/steveschaefer/2011/08/24/steve-jobs-steps-down-apple-shares-drop-5-after-hours/#2220e47240e3',
'http://money.cnn.com/2011/08/24/technology/apple_after_hours_shares/index.htm',
'http://money.cnn.com/2011/08/24/technology/steve_jobs_resigns/index.htm',
'https://seekingalpha.com/article/289692-how-the-steve-jobs-news-impacts-apple-in-the-long-term',
'http://money.cnn.com/2012/04/24/technology/apple-earnings/index.htm',
'http://money.cnn.com/2012/04/10/technology/apple_market_cap/index.htm',
'https://seekingalpha.com/article/527501-apples-delicious-quarterly-results',
'https://seekingalpha.com/article/536451-what-apple-is-doing-right-to-create-long-term-value',
'http://www.marketwatch.com/story/has-apple-lost-it-2012-10-10',
'https://www.forbes.com/sites/darcytravlos/2012/10/04/the-problem-for-apple-and-its-stock-price/#5b1a86e252d7',
'https://www.cnbc.com/id/49565882',
'https://seekingalpha.com/article/990291-apple-still-calm-before-the-storm',
'https://www.forbes.com/sites/briansolomon/2014/05/28/apple-brings-dr-dre-on-board-with-official-3-billion-beats-deal/',
'https://www.forbes.com/sites/zackomalleygreenburg/2014/05/30/evaluating-apples-3-billion-beats-buy-wins-losses-and-draws/',
'https://finance.yahoo.com/blogs/daily-ticker/apple-buying-beats-for--3-2-billion--smart-move-145319949.html',
'https://www.forbes.com/sites/nathanvardi/2014/05/12/tim-cook-dr-dre-and-the-battle-for-apples-cash/#523dd2eb3dd3',
'https://www.fool.com/investing/general/2014/06/16/heres-how-beats-electronics-will-benefit-apple-inc.aspx',
'http://money.cnn.com/2014/08/19/investing/apple-stock-record-high/index.html',
'https://www.fool.com/investing/general/2014/10/14/bose-vs-beats-the-nfl-tests-its-limits-with-its-he.aspx',
'https://investorplace.com/2014/11/aapl-buy-apple-stock-2015/#.WbWvZsbMz-Y',
'https://seekingalpha.com/article/2228023-the-real-reason-apple-is-acquiring-beats-for-3_2b-part-2',
'http://money.cnn.com/2015/04/28/technology/apple-stock-all-time-high/index.html',
'https://seekingalpha.com/article/3059826-apple-watch-success-not-limited-to-unit-sales',
'http://markets.businessinsider.com/news/stocks/apple-stock-price-three-reasons-after-wwdc-2017-6-1002071211',
'https://www.cnbc.com/2017/05/12/apples-next-iphone-will-break-the-1000-barrier-and-send-the-stock-soaring-goldman-says.html',
'http://fortune.com/2017/02/01/apple-shares-analysts/',
'https://investorplace.com/2017/02/why-apple-inc-aapl-stock-could-rise-to-150-upgrade/#.WbWygMbMz-Y',
'https://investorplace.com/2017/03/apple-inc-aapl-stock-rise-more-than-15/#.WbWyuMbMz-Y',
'https://www.thestreet.com/story/13336718/1/here-s-why-apple-aapl-stock-is-on-the-rise-today.html',
'https://www.nytimes.com/2017/06/06/technology/apple-reinvent-itself.html',
'https://www.apple.com/newsroom/2017/06/homepod-reinvents-music-in-the-home/',
'https://www.apple.com/newsroom/2017/01/app-store-shatters-records-on-new-years-day/',
'https://techcrunch.com/2017/12/11/apple-shazam-deal/',
'https://www.apple.com/newsroom/2017/08/apples-next-us-data-center-will-be-built-in-iowa/',
'https://www.apple.com/newsroom/2017/12/imac-pro-the-most-powerful-mac-ever-available-today/',
'https://www.usatoday.com/story/tech/news/2017/04/14/apple-joins-driverless-car-test-applicants/100467124/',
'https://www.forbes.com/sites/chrisversace/2017/06/06/apples-wwdc-2017-an-event-lacking-vision-from-a-company-without-a-visionary/#3928ddda1f7e',
'https://www.cnbc.com/2017/07/06/samsung-q2-earnings-preview-profit-apple.html',
'https://www.theguardian.com/technology/2017/apr/04/apple-planning-new-imac-mac-pro',
'https://www.cnbc.com/2017/11/03/us-stocks-apple-earnings-jobs-report.html',
'https://www.bloomberg.com/news/articles/2017-06-13/cook-says-apple-is-focusing-on-making-an-autonomous-car-system',
'https://www.forbes.com/sites/amitchowdhry/2017/11/07/apple-confirms-ios-11-1-autocorrect-bug-and-suggests-this-workaround/#1d0494746b7a',
'https://www.inc.com/bill-murphy-jr/apples-new-iphone-has-a-frustrating-bug-thats-driving-people-crazy.html',
'https://techcrunch.com/2017/11/07/apple-pay-cash-launches-in-beta-today-letting-you-send-and-receive-cash-in-messages/',
'https://www.nytimes.com/2017/09/06/business/apple-watch-red-sox-cheating.html',
'https://www.theverge.com/2017/10/10/16454312/apple-steven-spielberg-amazing-stories-original-content-deal',
'https://www.cnbc.com/2017/11/17/apple-just-delayed-its-homepod-smart-speaker-until-next-year.html',
'https://www.apple.com/newsroom/2017/06/macos-high-sierra-delivers-advanced-technologies-for-storage-video-and-graphics/',
'https://www.wired.com/2017/06/review-apple-ipad-pro-10-5-inch/',
'https://www.reuters.com/article/us-china-fortunteforum-apple/apples-cook-optimistic-that-apps-pulled-in-china-will-be-back-idUSKBN1E00MI',
'https://www.theguardian.com/technology/2017/jan/17/apple-ios-mac-app-store-prices-rise-25-per-cent-following-brexit',
'https://www.theverge.com/2017/11/6/16614038/apple-samsung-slide-to-unlock-supreme-court-120-million',
'https://www.theverge.com/2017/3/25/15056856/drake-more-life-apple-music-jimmy-iovine-streaming-numbers',
'https://www.theverge.com/2017/4/4/15175916/apple-mac-pro-redesign-plans',
'https://www.bloomberg.com/news/articles/2017-12-27/apple-ceo-cook-gets-74-bonus-boost-after-sales-profit-rebound',
'https://www.theverge.com/2017/8/16/16155600/apple-original-tv-content-1-billion-investment',
'https://www.cnbc.com/2017/12/21/apple-is-morgan-stanleys-top-tech-pick-for-2018.html',
'https://www.gsmarena.com/apple_pay_arrives_in_denmark_finland_sweden_and_uae-news-27882.php',
'https://standardoracle.com/2017/11/20/do-investors-want-to-take-risk-on-this-stock-apple-inc-aapl-2/',
'https://hitechgazette.com/apple-delays-the-launch-of-homepod-till-early-2018/',
'https://standardoracle.com/2017/09/18/stock-technical-performances-to-explore-apple-inc-aapl/',
'http://www.ozarksfirst.com/news/apple-apologizes-for-slowing-down-old-iphones-cuts-price-of-replacement-batteries/890462649',
'https://www.popsugar.com/news/Apple-Watch-LTE-Series-3-Details-44005001',
'http://bgr.com/2017/11/13/iphone-x-design-case-screen-protectors-ugly/',
'https://www.theguardian.com/business/2016/aug/30/apple-pay-back-taxes-eu-ruling-ireland-state-aid',
'https://techcrunch.com/2016/06/13/apple-finally-opens-siri-to-third-party-developers/',
'https://www.apple.com/newsroom/2016/06/apple-previews-ios-10-biggest-ios-release-ever/',
'https://www.bloomberg.com/news/articles/2016-08-30/apple-ordered-to-pay-up-to-14-5-billion-in-eu-tax-crackdown',
'https://qz.com/607396/google-has-officially-dethroned-apple-as-the-worlds-most-valuable-company/',
'http://fortune.com/2016/05/16/apple-stores-india/',
'https://www.theguardian.com/technology/2016/may/16/warren-buffett-apple-stocks-berkshire-hathaway',
'https://www.wired.com/2015/06/apple-builds-content-business-news-app/',
'https://www.forbes.com/sites/julianmitchell/2015/06/09/wwdc-2015-the-hidden-strategy-that-gives-apple-music-the-edge-over-its-competitors/#4a3834315464',
'https://www.marketwatch.com/story/apples-stock-tends-to-decline-after-wwdc-keynote-2015-06-08',
'http://money.cnn.com/2015/06/08/technology/tim-cook-women-apple/index.html',
'http://www.businessinsider.com/apple-wwdc-analysts-reaction-2015-6',
'http://money.cnn.com/2015/08/04/investing/apple-stock-down-iphone-china/index.html',
'http://money.cnn.com/2015/08/24/technology/apple-china/index.html',
'https://www.reuters.com/article/us-apple-stock/apple-shares-to-close-year-with-worst-performance-since-2008-idUSKBN0UE18A20151231',
'https://amigobulls.com/articles/apple-stock-has-the-magical-elixir-of-innovation-run-out',
'https://www.usatoday.com/story/money/markets/2015/12/15/apple-more-bad-news/77358628/',
'https://www.thestreet.com/story/13250111/1/cramer--this-is-why-you-hold-apple-stock.html',
'https://www.macrumors.com/2015/08/24/aapl-below-100-dow-jones-downturn/',
'https://www.fool.com/investing/general/2015/07/25/theres-no-way-apple-inc-sold-over-4-million-apple.aspx',
'https://www.cultofmac.com/316880/three-reasons-why-apple-will-be-the-worlds-first-trillion-dollar-company/',
'https://techcrunch.com/2015/03/09/apple-shares-shift-in-wake-of-new-product-announcements/',
'https://www.forbes.com/sites/briansolomon/2014/06/02/apple-shares-sink-after-wwdc-lacks-new-hardware/#1b8a1db450af',
'http://money.cnn.com/2014/06/06/investing/apple-stock-split/index.html',
'https://www.nasdaq.com/article/5-reasons-ill-never-sell-apple-stock-cm409836',
'https://www.fool.com/investing/general/2014/10/06/apple-pay-vs-google-wallet-why-apple-inc-will-win.aspx',
'http://abcnews.go.com/Business/apple-stock-aapl-crosses-500-twitter-norm-market/story?id=19958207',
'https://finance.yahoo.com/blogs/the-exchange/carl-icahn-multibillion-dollar-tweet-boosts-apple-stock-205938760.html',
'https://www.wired.com/2013/08/icahn-apple-tweet/',
'http://money.cnn.com/2013/01/24/investing/apple-stock/index.html',
'https://www.beaumontenterprise.com/news/article/Exxon-surpasses-Apple-as-most-valuable-company-4224406.php',
'https://www.cnet.com/news/apple-no-longer-the-worlds-most-valuable-company/',
'http://business.time.com/2013/10/04/two-years-after-steve-jobs-death-is-apple-a-different-company/',
'http://buzz.money.cnn.com/2013/05/21/apple-tim-cook-congress/',
'https://www.marketwatch.com/story/10-reasons-to-buy-apple-stock-now-2013-12-11',
'https://www.forbes.com/sites/darcytravlos/2013/04/21/do-i-buy-apple-on-monday/#74dcfac620f6',
'http://www.businessinsider.com/fidelity-sells-apple-stock-aapl-2013-6',
'http://www.nytimes.com/2013/04/19/technology/after-apples-rise-a-bruising-fall.html',
'https://www.thestreet.com/story/11815657/1/apple-just-like-a-stinking-penny-stock.html',
'https://www.usatoday.com/story/money/columnist/krantz/2013/04/17/apple-stock-growth-fall/2057925/',
'http://www.businessinsider.com/david-trainer-on-apples-stock-2013-5',
'https://investorplace.com/2013/10/apple-stock-aapl-microsof-msft/#.WpyS4maZPjA',
'https://www.barrons.com/articles/BL-FUNDSB-12145',
'https://www.macrumors.com/2013/12/05/apples-stock-hits-new-52-week-high-as-roller-coaster-2013-nears-its-end/',
'http://money.cnn.com/2013/12/30/technology/tim-cook-pay/index.html',
'https://www.slashgear.com/apples-tim-cook-demands-bigger-stock-risk-and-costs-himself-4m-30310366/',
'https://www.usatoday.com/story/money/markets/2013/04/18/apple-crash-steve-jobs/2094379/',
'http://www.businessinsider.com/apple-stock-low-2013-4',
'http://www.nytimes.com/2013/02/09/business/following-a-herd-of-bulls-on-apples-stock.html',
'https://etfdailynews.com/2013/03/25/apple-inc-aapl-can-the-stock-be-great-again/',
'https://www.fool.com/investing/general/2013/06/19/warren-buffetts-simple-advice-about-apple-stock.aspx',
'https://www.marketwatch.com/story/apple-and-the-iwatch-the-next-frontier-2013-02-12',
'https://www.fool.com/investing/general/2013/04/25/1-hidden-reason-why-apples-margins-are-falling.aspx',
'http://www.businessinsider.com/a-simple-chart-that-explains-the-drop-in-apples-stock-2013-1',
'https://www.theregister.co.uk/2013/10/08/billionaire_dumps_apple_stock_because_steve_jobs_was_really_awful_guy/',
'https://www.fool.com/investing/general/2013/07/10/why-im-doubling-down-on-apple-stock.aspx',
'http://appleinsider.com/articles/13/10/24/aapl-shares-regain-highs-from-early-2013-many-apple-analysts-targets-now-underwater'
          ]
URLListGE = [
'https://investorplace.com/2017/07/general-electric-company-ge-stock-survive/#.WqTBGWaZPjA',
'https://www.thestreet.com/story/14237190/1/get-out-of-general-electric-ahead-of-the-quarter.html',
'https://finance.yahoo.com/news/ge-report-q2-earnings-pull-134501576.html',
'https://investorplace.com/2017/01/general-electric-company-ge-stock-history-winning/#.WqTBI2aZPjA',
'https://www.barrons.com/articles/general-electric-much-ado-about-nothing-1500472930',
'https://money.usnews.com/investing/stock-market-news/articles/2017-11-15/general-electric-company-ge-stock-dividend',
'https://www.thestreet.com/story/14395148/1/ge-ceo-shows-faith-in-stock-with-million-dollar-purchase.html',
'https://www.reuters.com/article/us-ge-stocks-ceo/ge-shares-rise-after-ceo-flannery-buys-stock-idUSKBN1DH1VY',
'https://investorplace.com/2017/07/general-electric-company-ge-stock-no-longer-worth/#.WqTBMGaZPjA',
'https://investorplace.com/2017/04/general-electric-company-ge-stock-is-a-buy-at-last/#.WqTBM2aZPjA',
'https://investorplace.com/2017/09/heres-when-youll-want-to-buy-general-electric-company-ge-stock/#.WqTBNGaZPjA',
'https://seekingalpha.com/article/4060287-general-electric-36-percent-total-return-opportunity-just-materialized',
'https://www.thestreet.com/story/14297502/1/ge-stock-suffers-as-jpmorgan-lowers-outlook-even-further.html',
'https://www.cnbc.com/2017/09/07/jpmorgan-says-ges-fundamentals-are-worse-than-we-think.html',
'https://investorplace.com/2017/08/general-electric-company-ge-stock-permanently-stuck-20s/#.WqTBQmaZPjA',
'https://www.bizjournals.com/boston/news/2017/08/10/with-stock-buy-ges-new-ceo-puts-millions-behind-a.html',
'https://www.timesunion.com/7day-business/article/GE-s-new-CEO-buys-2-7-million-in-stock-for-401-k-11750062.php',
'https://investorplace.com/2017/05/general-electric-company-ge-stock-52-week-lows/#.WqTBTGaZPjA',
'https://investorplace.com/2017/05/general-electric-company-ge-stock-tech-play/#.WqXltWaZPjA',
'https://investorplace.com/2017/08/john-flannery-cant-budge-general-electric-company-ge-stock-with-words/#.WqXlt2aZPjA',
'https://investorplace.com/2017/09/general-electric-corporation-ge-short/#.WqXluWaZPjA',
'https://www.forbes.com/sites/investor/2017/09/14/why-now-is-a-good-time-to-buy-general-electric/#c19a84b53055',
'https://investorplace.com/2017/06/general-electric-company-ge-stock-far-more-dangerous/#.WqXlvWaZPjA',
'https://www.fool.com/investing/2017/06/16/its-time-for-general-electric-to-bet-big-on-solar.aspx',
'https://investorplace.com/2017/06/general-electric-company-ge-stock-catalyst-missed/#.WqXlwWaZPjA',
'https://www.thestreet.com/story/14419577/1/ge-stock-is-awful.html',
'https://investorplace.com/2017/06/how-general-electric-company-ge-stock-can-return-25/#.WqXlymaZPjA',
'https://investorplace.com/2017/12/ge-stock-nowhere-fast/#.WqXl0WaZPjA',
'https://investorplace.com/2017/01/abandon-general-electric-company-ge-stock-after-q4-earnings/#.WqXl1WaZPjA',
'https://investorplace.com/2017/02/general-electric-company-ge-stock-isnt-worth-3-2-yield/#.WqXl12aZPjA',
'https://investorplace.com/2017/08/general-electric-company-ge-stock-dog-dow/#.WqXl22aZPjA',
'https://investorplace.com/2017/05/its-time-general-electric-company-ge-stock-died/#.WqXl3maZPjA',
'https://investorplace.com/2017/03/general-electric-company-ge-stock-powerhouse/#.WqXl4WaZPjA',
'https://investorplace.com/2016/12/general-electric-company-ge-stock-isnt-boring-anymore/#.WqXl72aZPjA',
'https://investorplace.com/2016/12/general-electric-company-ge-good-company-terrible-stock/#.WqXl82aZPjA',
'https://investorplace.com/2016/11/general-electric-company-ge-stock-trumps-america/#.WqXl_2aZPjA',
'https://investorplace.com/2016/10/general-electric-company-ge-stock-solid-long-term-buy/#.WqXmAmaZPjA',
'https://www.fool.com/investing/2016/10/09/a-simple-reason-to-consider-buying-general-electri.aspx',
'https://investorplace.com/2016/10/general-electric-ge-stock-better/#.WqXmCWaZPjA',
'https://seekingalpha.com/article/4010675-general-electric-company-hit-60-share-within-year',
'https://investorplace.com/2016/08/general-electric-stock-ge/#.WqXmE2aZPjA',
'http://fortune.com/2016/04/22/general-electric-ge-stock-earnings/',
'https://investorplace.com/2016/02/ge-stock-general-electric-dividends/#.WqXmH2aZPjA',
'https://amigobulls.com/articles/is-general-electric-company-stock-a-buy-ahead-of-its-q2-earnings',
'https://www.benzinga.com/trading-ideas/short-ideas/16/09/8432173/ge-stock-is-breaking-down',
'https://www.thestreet.com/story/13613324/1/ge-stock-is-the-lsquo-chart-of-the-day-rsquo.html',
'http://fortune.com/2016/05/12/general-electric-sell-jpmorgan/',
'https://www.thestreet.com/story/13328925/1/don-t-let-go-of-ge-despite-21-billion-synchrony-swap-offer.html',
'https://etfdailynews.com/2015/10/19/general-electric-company-ge-diversification-spurs-the-stock/',
'https://www.fool.com/investing/general/2015/01/20/volatile-oil-prices-general-electric-earnings.aspx',
'https://www.reuters.com/article/us-ge-valuation/general-electric-valuation-hits-lofty-levels-after-stock-surge-idUSKBN0TX2F020151214',
'https://investorplace.com/2015/02/general-electric-stock-general-electric-companyge-stock/#.WqXmaGaZPjA',
'https://www.cnbc.com/2015/10/05/cramer-peltz-could-double-ges-stock.html',
'http://money.cnn.com/2015/08/25/investing/ge-mystery-stock-plunge-china/index.html',
'https://www.thestreet.com/story/12890967/1/why-general-electric-ge-stock-is-down-today.html'
]
URLListTSLA = [
'http://www.businessinsider.com/tesla-stock-price-elon-musk-short-sellers-profit-from-decline-2017-11',
'http://fortune.com/2017/11/17/tesla-stock-roadster-semi-truck/',
'https://www.cnbc.com/2017/08/03/wall-street-stunned-on-teslas-stock-surge-skeptical-on-model-3-ramp.html',
'https://money.usnews.com/investing/stock-market-news/articles/2017-08-02/tesla-tsla-stock-earnings',
'https://techcrunch.com/2017/07/05/tesla-drops-7-after-goldman-sachs-says-the-stock-is-worth-180/',
'https://www.marketwatch.com/story/teslas-stock-tumbles-into-correction-territory-but-analysts-sees-new-highs-ahead-2017-07-05',
'https://investorplace.com/2017/07/tesla-inc-tsla-stock-will-hit-400-as-model-3-hype-turns-reality/#.WqXzrGaZPjA',
'https://www.cnbc.com/2017/02/23/tesla-stock-drops-after-earnings-but-analysts-see-new-highs-ahead.html',
'http://www.latimes.com/business/autos/la-fi-hy-tesla-1q-production-20170403-story.html',
'https://www.reuters.com/article/us-tesla-stocks/elon-musk-mocks-short-sellers-after-tesla-stock-surge-idUSKBN17522H',
'https://www.marketwatch.com/story/tesla-surpasses-ford-to-become-no-2-us-car-maker-2017-04-03',
'https://money.usnews.com/investing/stock-market-news/articles/2017-12-01/tesla-inc-tsla-stock',
'https://www.fool.com/investing/2017/01/04/the-simple-reason-i-wont-buy-tesla-motors-stock.aspx',
'http://fortune.com/2017/07/08/tesla-stock-delivery/',
'https://www.forbes.com/sites/petercohan/2017/07/07/why-you-should-sell-tesla-on-weeks-9-billion-plunge/#44e0761ad5a6',
'https://www.fool.com/investing/2017/07/06/why-tesla-inc-stock-is-down-again-today.aspx',
'http://www.businessinsider.com/chart-explains-why-teslas-stock-price-insane-territory-2017-4',
'http://fortune.com/2017/01/19/tesla-elon-musk-donald-trump-morgan-stanley/',
'https://www.bloomberg.com/news/articles/2017-04-10/tesla-passes-gm-as-musk-s-carmaker-becomes-america-s-top-valued',
'https://www.thestreet.com/story/14371576/1/tesla-stock-drops-amid-mixed-q3-model-3-bottlenecks.html',
'https://www.reuters.com/article/us-tesla-results/tesla-stock-falls-on-model-3-delays-biggest-ever-quarterly-loss-idUSKBN1D15TP',
'https://www.forbes.com/sites/chuckjones/2017/11/01/if-teslas-model-3-news-is-bad-look-out-below/#3880eab668b6',
'https://investorplace.com/2017/10/tesla-inc-tsla-stock-500-per-share/#.WqXz_GaZPjA',
'https://www.thestreet.com/story/14164811/1/tesla-s-stock-could-pop-a-quick-26-because-of-its-awesome-new-model-3.html',
'https://electrek.co/2017/06/07/tesla-tsla-all-time-fortune-500-list/',
'https://investorplace.com/2017/06/knock-down-tesla-inc-tsla-stock/#.WqX0DGaZPjA',
'https://www.investopedia.com/news/tesla-stock-analysts-divided-after-low-model-3-deliveries/',
'https://www.investors.com/news/technology/tesla-stock-falls-on-model-3-production-miss-due-to-bottlenecks/',
'https://www.cnbc.com/2017/11/02/tesla-stock-best-performing-tech-name-in-next-5-years-analyst-says.html',
'http://www.latimes.com/business/la-fi-hy-tesla-tax-subsidy-20171102-story.html',
'https://www.fool.com/investing/2017/07/28/why-the-model-3-is-so-important-for-tesla-inc.aspx',
'https://www.cnbc.com/2017/07/17/elon-musk-teslas-stock-price-is-higher-than-we-deserve-right-now.html',
'https://www.cnbc.com/2017/02/09/tesla-stock-jumps-on-word-of-model-3-production-preparations.html',
'https://www.marketwatch.com/story/tesla-shares-rise-as-first-model-3-rolls-off-production-line-2017-07-10',
'http://www.businessinsider.com/tesla-stock-faces-challenges-after-semi-roadster-reveal-2017-11',
'https://www.investopedia.com/news/how-tesla-stock-benefits-trump-administration-tsla-f/',
'http://www.businessinsider.com/tesla-stock-model-3-2016-4',
'https://www.investors.com/news/technology/tesla-stock-lifts-as-model-3-lines-stretch-how-many-reservations/',
'https://www.usatoday.com/story/money/cars/2016/04/01/tesla-motors-model-3-pre-orders/82506056/',
'https://www.reuters.com/article/us-tesla-model3/tesla-buoyed-by-strong-model-3-orders-may-need-more-cash-idUSKCN0WY3JK',
'https://money.usnews.com/investing/articles/2016-04-01/the-model-3-may-be-tesla-motors-biggest-test-yet-tsla',
'http://www.valuewalk.com/2016/10/tesla-motors-inc-tsla-no-energy/',
'https://money.usnews.com/investing/articles/2016-10-20/all-tesla-motors-inc-tsla-cars-will-now-feature-self-driving-hardware',
'https://www.thestreet.com/story/13626717/1/tesla-tsla-stock-falls-in-after-hours-trading-on-model-s-investigation.html',
'https://www.usatoday.com/story/money/columnist/krantz/2016/04/04/ask-matt-how-high-can-tesla-stock-go/82528438/',
'http://money.cnn.com/2016/02/10/news/companies/tesla-earnings/index.html',
'http://www.businessinsider.com/tesla-earnings-q1-2016-2016-5',
'https://www.cnbc.com/2016/06/22/solarcity-stock-soars-as-wall-street-reacts-to-tesla-motors-28-billion-offer.html',
'http://www.businessinsider.com/tesla-made-offer-to-buy-solar-city-2016-6',
'http://www.bbc.com/news/technology-36765823',
'http://www.latimes.com/business/autos/la-fi-hy-musk-master-plan-20160714-snap-story.html',
'http://time.com/money/4279284/tesla-stock-price-surge/',
'http://money.cnn.com/2016/04/01/news/companies/tesla-model-3-stock-price/index.html',
'https://investorplace.com/2016/11/tesla-motors-inc-tsla-stock-solar-roof-ipmedia/#.WqX0_GaZPjA',
'http://fortune.com/2016/05/18/goldman-sachs-tesla-musk/',
'https://www.forbes.com/sites/greatspeculations/2016/11/21/tesla-can-boost-its-stock-price-by-50-if-it-meets-its-production-targets/#5b95729d475a',
'https://www.cnbc.com/2016/05/03/tesla-stock-is-not-a-buy-no-matter-what-earnings-say-commentary.html',
'https://www.thestreet.com/story/13652719/1/mobileye-mbly-stock-continues-to-decline-on-tesla-split.html',
'http://www.businessinsider.com/2017-tesla-predictions-2016-12',
'https://www.profitconfidential.com/stock/tesla-motors-inc-tsla-stock-chart-bullish/',
'http://fortune.com/2016/02/05/heres-whats-behind-teslas-long-slow-stock-slide/',
'https://investorplace.com/2015/05/tesla-powerwall-buy-tsla-stock/#.WqX1WWaZPjA',
'https://www.fool.com/investing/general/2015/08/01/teslas-referral-program-smart-move-or-sign-of-desp.aspx',
'https://www.benzinga.com/analyst-ratings/analyst-color/15/08/5769540/adam-jonas-why-tesla-stock-is-worth-465',
'https://www.bloomberg.com/news/articles/2015-08-17/morgan-stanley-massively-hikes-price-target-on-tesla-says-stock-could-almost-double',
'http://money.cnn.com/2015/01/13/investing/tesla-stock-gm-bolt/index.html',
'http://www.businessinsider.com/why-tesla-always-overpromises-and-underdelivers-2015-10',
'https://www.recode.net/2015/10/20/11619780/tesla-stock-is-falling-fast-after-consumer-reports-rescinds',
'https://www.engadget.com/2015/10/20/consumer-reports-tesla-model-s-reliability/',
'http://www.businessinsider.com/report-consumer-reports-pulls-recommendation-for-tesla-model-s-2015-10',
'https://qz.com/528863/tesla-shares-are-sinking-after-consumer-reports-raised-reliability-concerns-about-the-model-s/',
'https://www.investopedia.com/articles/active-trading/040215/now-good-time-invest-tesla.asp',
'https://www.forbes.com/sites/briansolomon/2015/03/30/elon-musk-boots-tesla-stock-with-major-new-product-tweet-tease/#b50bcec3f9a7',
'http://money.cnn.com/2015/09/30/investing/tesla-model-x-suv-reviews-stock/index.html',
'https://www.cnbc.com/2015/11/06/why-tesla-stock-may-grow-4x-billionaire-ron-baron.html',
'http://www.businessinsider.com/morgan-stanley-tesla-price-target-model-x-2015-10',
'https://mashable.com/2014/10/10/tesla-d-stock/#gDTi40d_buqu',
'http://www.businessinsider.com/tesla-electric-cars-could-take-years-to-outpace-gas-2014-8',
'http://money.cnn.com/2014/12/08/investing/tesla-stock-oil-prices/index.html',
'https://www.huffingtonpost.com/2014/09/02/tesla-stock-price_n_5755608.html',
'http://www.cbc.ca/news/business/tesla-stock-soars-after-elon-musk-pledges-100-000-cars-a-year-1.2725186',
'https://www.huffingtonpost.com/2014/12/16/tesla-stock-price_n_6336412.html'
]
URLListAMZN = [
'https://www.cnbc.com/2017/06/16/after-its-stock-pop-amazon-will-get-whole-foods-essentially-for-free.html',
'https://www.theatlantic.com/technology/archive/2017/06/when-exactly-does-amazon-become-a-monopoly/530616/',
'https://money.usnews.com/investing/stock-market-news/articles/2017-11-26/amazon-com-inc-amzn-stock-cyber-monday',
'https://www.investors.com/research/ibd-industry-themes/amazon-shares-hit-record-high-as-cyber-monday-sales-kick-in/',
'https://investorplace.com/2017/11/cyber-monday-sales-give-amazon-com-inc-stock-power/',
'http://fortune.com/2017/07/10/amazon-prime-day-2017-stock-price/',
'https://www.investors.com/news/technology/heres-what-you-need-to-know-about-amazons-prime-day-bonanza/',
'https://www.theverge.com/2017/7/27/16052792/amazon-q2-2017-earnings-jeff-bezos-worlds-richest-person-title',
'https://www.cnbc.com/2017/06/19/amazon-stock-jumps-to-new-all-time-high-of-1017.html',
'https://www.thestreet.com/story/14323008/1/amazon-stock-gains-jpmorgan.html',
'https://www.fool.com/investing/2017/09/28/amazon-doubles-down-on-alexa-with-new-devices.aspx',
'http://markets.businessinsider.com/news/stocks/amazon-stock-price-is-rising-after-announcing-tons-of-new-hardware-2017-9-1002667999',
'https://www.cnbc.com/2017/05/30/amazon-shares-hit-1000.html',
'https://www.seattletimes.com/business/amazon/amazons-stock-tops-1000-for-the-first-time-2/',
'https://www.usatoday.com/story/tech/news/2017/05/30/amazon-hits-1000-what-keep-2000/102316448/',
'https://money.usnews.com/investing/stock-market-news/articles/2017-12-29/amazon-com-inc-amzn-trump',
'http://www.newsweek.com/amazon-stocks-fall-5-bn-after-single-tweet-trump-651675',
'http://www.businessinsider.com/amazon-q3-earnings-report-2017-10',
'http://fortune.com/2017/10/27/amazon-microsoft-alphabet-google-intel-stock/',
'https://www.cnbc.com/2017/10/27/tech-stocks-leaping-higher-after-blowout-earnings-from-amazon-alphabet-and-microsoft.html',
'https://www.mercurynews.com/2017/10/27/us-stocks-are-mixed-as-tech-soars-retail-and-energy-sink/',
'https://www.investors.com/news/nike-earnings-late-look-for-chatter-on-amazon-new-strategy/',
'https://seekingalpha.com/article/4117920-amazons-q3-expensive-stock-still-worth-owning',
'https://www.seattletimes.com/business/amazon/amazon-worth-more-than-500-billion-as-stock-continues-its-climb/',
'https://www.geekwire.com/2017/amazon-stock-surges-13-earnings-report-making-jeff-bezos-worlds-richest/',
'https://www.investors.com/news/technology/amazon-boasts-of-strong-holiday-sales-ahead-of-digital-day/',
'https://www.investopedia.com/news/amazons-prime-day-too-big-24-hours-stock-rally-expected/',
'https://www.nasdaq.com/article/will-amazon-amzn-stock-rally-ahead-of-prime-day-cm809581',
'http://www.heraldnet.com/business/with-surging-amazon-stock-bezos-now-worth-more-than-100b/',
'https://www.investors.com/news/technology/amazon-makes-big-middle-east-play-with-souq-com-acquisition/',
'https://www.zacks.com/stock/news/273258/heres-the-reason-for-amazons-recent-stock-pullback',
'http://markets.businessinsider.com/news/stocks/amazon-stock-price-faces-one-big-problem-for-food-delivery-and-its-why-it-bought-whole-foods-2017-8-1002276390',
'https://www.cnbc.com/2016/07/11/amazon-prime-day-is-no-joke-for-investors-as-stock-soars.html',
'https://www.investors.com/news/technology/amazon-introduces-amazon-go-retail-stores-no-checkout-no-lines/',
'https://investorplace.com/2016/12/amazon-com-inc-amzn-stock-800-holiday/',
'http://www.businessinsider.com/amazon-earnings-q1-2016-2016-4',
'https://www.geekwire.com/2016/amazon-stock-nears-time-high-record-earnings-tech-giant-valued-310b/',
'http://time.com/money/4312379/amazon-q1-earnings-investor-confidence/',
'http://blogs.marketwatch.com/thetell/2016/04/28/amazon-earnings-expected-to-show-a-return-to-profit-live-blog/',
'http://money.cnn.com/2016/10/27/technology/amazon-earnings/index.html',
'https://www.marketwatch.com/story/amazons-stock-chart-warns-this-years-amazing-ride-may-be-over-2016-10-28',
'https://techcrunch.com/2016/07/28/amazon-shatters-earnings-expectations/',
'https://www.thestreet.com/story/13656806/1/amazon-com-amzn-stock-falls-in-after-hours-trading-despite-q2-earnings-beat.html',
'https://www.investors.com/news/technology/amazon-poised-to-become-no-1-u-s-apparel-retailer-by-2017/',
'https://www.investors.com/news/technology/amazon-calls-prime-day-biggest-day-ever-with-strong-global-sales/',
'https://www.investors.com/news/technology/amazon-launches-youtube-competitor-on-prime-video-platform/',
'http://money.cnn.com/2016/05/02/investing/amazon-stock-warren-buffett-jeff-bezos/index.html',
'http://www.businessinsider.com/amazon-closes-record-high-stock-price-2016-9',
'https://investorplace.com/2016/03/amazon-com-inc-amzn-stock-heading-lower/',
'https://www.thestreet.com/story/13842305/1/amazon-com-amzn-stock-higher-price-target-raised-at-morgan-stanley-suntrust.html',
'https://investorplace.com/2016/07/amzn-stock-amazon-com-inc-earnings/',
'https://www.marketwatch.com/story/three-reasons-why-amazons-stock-may-be-unstoppable-2016-06-01',
'http://www.businessinsider.com/amazon-greatest-strength-in-two-sentences-2016-10',
'https://www.geekwire.com/2015/amazon-com-customers-not-happy-with-initial-prime-day-selection/',
'http://fortune.com/2015/07/24/amazon-stock-surge/',
'https://www.forbes.com/forbes/welcome/?toURL=https://www.forbes.com/sites/ryanmac/2015/07/23/amazon-second-quarter-earnings-2015/&refURL=https://www.google.com/&referrer=https://www.google.com/',
'https://www.marketwatch.com/story/amazon-analysts-bullish-on-the-stock-after-big-earnings-beat-2015-07-24',
'https://www.cnbc.com/2015/07/24/amazon-shares-surge-20-percent-market-cap-surpasses-wal-mart.html',
'https://www.nytimes.com/2015/07/24/technology/amazon-earnings-q2.html',
'https://www.geekwire.com/2015/amazon-stock-14-percent-investors-rally-around-fourth-quarter-profits/',
'http://www.businessinsider.com/why-amazons-stock-is-going-nuts-2015-1',
'http://money.cnn.com/2015/01/29/investing/amazon-earnings/index.html',
'http://www.slate.com/blogs/moneybox/2015/01/29/amazon_tops_q4_earnings_expectations_wall_street_is_actually_happy_with.html',
'https://www.seattletimes.com/business/amazon/amazon-shares-soar-after-company-posts-surprise-profit-6/',
'https://www.washingtonpost.com/news/business/wp/2015/10/22/amazon-stock-is-surging-thanks-to-surprisingly-upbeat-earnings/?utm_term=.51fdd1c67771',
'https://finance.yahoo.com/news/why-amazon-s-stock-is-really-flying--hint--it-s-not-the-cloud-144934941.html',
'https://www.geekwire.com/2015/amazons-shares-up-50-this-year-as-stock-continues-to-soar-on-the-outlook-for-prime/',
'https://www.marketwatch.com/story/amazon-crushes-black-friday-weekend-with-record-sales-of-its-own-devices-2015-12-01',
'https://www.marketwatch.com/story/amazon-is-now-an-unstoppable-stock-2015-10-26',
'https://www.marketwatch.com/story/analysts-are-talking-500-for-amazon-shares-2015-04-24',
'https://www.thestreet.com/story/13382999/1/it-s-time-to-sell-your-amazon-com-stock-this-technical-chart-reveals.html',
'https://www.theguardian.com/technology/2015/jul/23/amazon-stock-surge-makes-ceo-jeff-bezos-7bn-richer-in-45-minutes',
'https://www.forbes.com/sites/petercohan/2015/12/31/3-reasons-amazon-stock-will-soar-in-2016/#114ffd507117',
'https://www.huffingtonpost.com/2014/10/23/amazon-stock-tanks-sales_n_6037630.html',
'https://www.forbes.com/sites/ryanmac/2014/10/23/amazon-third-quarter-2014-earnings/#28c07ee71031',
'https://investorplace.com/2014/12/amazon-stock-amzn-timeline-2014/',
'https://newrepublic.com/article/120464/amazons-revenue-growth-has-fallen-making-investors-nervous',
'https://www.marketwatch.com/story/7-reasons-amazons-stock-is-going-nowhere-in-2014-2014-05-16',
'https://www.fool.com/investing/high-growth/2014/04/27/amazoncom-continues-to-be-the-most-innovative-reta.aspx',
'https://www.forbes.com/sites/greatspeculations/2013/01/30/amazons-third-party-sellers-drive-margins-and-stock-higher/#16e488f952b0',
'https://www.wired.com/2013/01/amazon-stock-hits-all/'
]
URLListNVDA = [
'https://cnafinance.com/nvidia-corporation-nvda-stock-falling-on-trade-war-news/18897',
'https://investorplace.com/2018/03/go-long-the-nvidia-corporation-nvda-stock-dip-before-it-closes-and-with-certainty/',
'https://investorplace.com/2018/03/nvidia-stock-has-30-percent-upside-from-current-levels/',
'https://www.fool.com/investing/2018/03/19/10-reasons-to-buy-nvidia-stock-and-hold-on-for-the.aspx',
'https://seekingalpha.com/article/4160331-nvidia-ai-will-drive-growth',
'https://finance.yahoo.com/news/why-nvidia-corporation-nvda-5-090109792.html',
'https://stocknewsjournal.com/2018/03/16/why-investors-remained-buoyant-on-nvidia-corporation-nvda-fireeye-inc-feye/',
'http://productioninvestments.com/notable-eye-catching-stocks-nvidia-corporation-nasdaq-nvda/',
'https://finance.yahoo.com/news/apos-why-nvidia-nvda-stock-155203981.html',
'https://investorplace.com/2017/11/nvidia-corporation-nvda/',
'https://www.nasdaq.com/article/buy-nvidia-nvda-stock-ahead-of-2nd-quarter-earnings-jefferies-cm829962',
'https://investorplace.com/2017/10/nvidia-nvda-stock-trading-200/',
'https://www.nasdaq.com/article/5-reasons-to-add-nvidia-nvda-stock-to-your-portfolio-now-cm833916',
'https://finance.yahoo.com/news/buy-nvidia-nvda-stock-now-220810467.html',
'https://investorplace.com/2017/07/should-you-chase-nvidia-corporation-nvda-stock-here/',
'https://investorplace.com/2017/06/buy-nvidia-corporation-nvda-stock-if-youre-sane/',
'https://investorplace.com/2017/05/nvidia-corporation-nvda-stock-will-hit-200-before-amd-hits-16/',
'https://investorplace.com/2017/07/nvidia-corporation-nvda-stock-worth-buying-on-decline-toward-120/',
'https://investorplace.com/2017/09/3-signs-that-nvidia-corporation-nvda-stock-is-oversold/',
'https://www.thestreet.com/story/14324835/1/nvidia-stock-could-rally.html',
'https://investorplace.com/2017/09/nvda-stock-still-king/',
'https://investorplace.com/2017/06/nvidia-why-nvda-stock-is-heading-to-180/',
'https://investorplace.com/2017/10/nvidia-corporation-nvda-stock-jumps-record-high/',
'https://www.valuewalk.com/2017/05/nvidia-corporation-nvda-stock-tear-artificial-intelligence/',
'https://investorplace.com/2017/07/nvidia-corporation-nvda-welcome-to-the-era-of-ai/',
'https://investorplace.com/2017/10/nvda-stock-caution-signals/',
'https://investorplace.com/2017/05/nvidia-corporation-nvda-stock-bubble-is-about-to-pop/',
'https://www.nasdaq.com/article/why-nvidia-nvda-might-be-a-diamond-in-the-rough-cm842851',
'https://amigobulls.com/articles/2017-06-16-nvidia-corporation-nvda-stock-is-still-a-great-buy-here-is-why',
'https://investorplace.com/2017/02/dont-count-nvidia-corporation-nvda-stock-just-yet/',
'https://investorplace.com/2017/07/3-reasons-nvidia-corporation-nvda-stock-bullish-case/',
'https://investorplace.com/2017/08/is-nvidia-corporation-nvda-stock-in-buy-territory/',
'https://investorplace.com/2017/05/why-nvidia-corporation-nvda-stock-will-hit-130/',
'https://investorplace.com/2016/12/is-nvidia-corporation-nvda-stock-for-real/',
'https://seekingalpha.com/article/3906826-nvidias-automotive-strategy-pays',
'https://www.investopedia.com/news/nvidia-corporation-surges-after-q3-earnings-nvda/',
'https://investorplace.com/2016/12/nvidia-stock-nvda-megatrends-power-more-gains/',
'https://seekingalpha.com/article/4025086-nvidia-easily-double-tripling',
'https://seekingalpha.com/article/4033450-nvidia-expect-strong-gains-2020',
'https://marketrealist.com/2016/10/nvidias-stock-continues-outperform-market-peers',
'https://finance.yahoo.com/news/why-nvidia-nvda-stock-might-124012807.html',
'https://amigobulls.com/articles/why-you-should-buy-nvidia-corporation-nvda-stock-on-dips',
'https://www.investopedia.com/stock-analysis/122916/heres-why-nvidia-dropped-and-why-not-worry-nvda.aspx',
'https://247wallst.com/technology-3/2016/12/28/nvidia-is-americas-most-successful-company-of-2016/',
'https://www.thestreet.com/story/13658102/1/nvidia-nvda-could-be-the-intel-of-artificial-intelligence-ritholtz-s-ceo-brown-tells-cnbc.html',
'https://www.marketwatch.com/story/nvidia-stock-rockets-to-record-high-following-earnings-shocker-2016-11-11',
'https://www.investopedia.com/stock-analysis/061615/3-reasons-nvidia-stock-could-rise-nvda.aspx',
'https://investorplace.com/2015/08/nvidia-stock-nvda/',
'http://www.businessinsider.com/r-nvidia-revenue-rises-45-percent-on-strong-demand-for-car-chips-2015-8',
'https://www.investopedia.com/stock-analysis/040715/2-huge-growth-opportunities-nvidia-nvda.aspx',
'https://www.fool.com/investing/general/2015/01/29/nvidia-wants-to-power-the-drone-market.aspx',
'https://wccftech.com/nvidia-nasdaq-nvda-q3-fy16-financial-results/',
'https://investorplace.com/2015/12/nvidia-buy-nvdas-breakout-bang-dollar/',
'https://www.thestreet.com/story/13191279/1/pre-market-activity-shows-gains-for-nvidia-nvda.html',
'https://seekingalpha.com/article/2288203-is-there-more-upside-left-in-nvidia',
'https://www.fool.com/investing/general/2014/04/08/why-nvidia-will-continue-to-dominate-the-pc-gaming.aspx',
'https://investorplace.com/2017/09/best-stocks-nvidia-corporations-nvda-stock-run/',
'https://www.nasdaq.com/article/nvidia-corporation-nvda-looks-good-stock-adds-53-in-session-cm876332',
'https://investorplace.com/2017/11/nvidia-corporation-nvda-stock-upside/',
'https://investorplace.com/2017/11/why-nvidia-corporation-stock-is-unstoppable/',
'https://investorplace.com/2017/03/nvidia-corporation-nvda-stock-downright-dangerous/',
'https://www.benzinga.com/analyst-ratings/analyst-color/17/11/10805616/there-could-be-18-upside-left-in-nvidia-analyst-says',
'https://amigobulls.com/articles/2017-05-26-more-reasons-to-buy-nvidia-corporation-nvda-stock',
'https://investorplace.com/2017/06/has-nvidia-corporation-nvda-stock-finally-reached-sell-territory/',
'https://www.marketwatch.com/story/nvidias-stock-rocked-after-analysts-say-its-time-to-sell-2017-02-23',
'https://investorplace.com/2017/10/nvda-stock-200-share/',
'https://investorplace.com/2017/03/nvidia-corporation-nvda-stock-wont-stop-growing/'
]

#### Main Procedure Block

In [25]:
def BusinessDay (strday):
    lyst = list(StockHistory['timestamp'])
    if (strday in lyst):
        return True
    else:
        return False

In [205]:
stockname = "AAPL"
StockHistory = CollectData(stockname)

for i in range(0,len(URLListAAPL)-1):
    url = URLListAAPL[i]
    articletext, articledate = URLtoTextSummary(url)
    
    #Sentiment Scores
    ListedText = articletext.split() #Turn into list
    pos, neg = getSentiment(ListedText)
    
    #Filter System
    if (articledate is None) or (isinstance(articledate, unicode)) or (len(StockHistory[StockHistory['timestamp'] == articledate.strftime('%Y-%m-%d')].index.tolist()) == 0): #If no date or article written on weekend:
        continue #Forget this article, move on
    
    #Days Old
    daysOld = randint(0,5)
    today = articledate + timedelta(days=daysOld)
    strtoday = today.strftime('%Y-%m-%d')
    while BusinessDay(strtoday) == False:
        today = today - timedelta(days=1)
        strtoday = today.strftime('%Y-%m-%d')
    daysOld = (today - articledate).days
    
    #Prices
    a = SearchCSVForDate(strtoday)
    pricetoday = StockHistory.get_value(a,"adjusted_close")
    priceonedayago = StockHistory.get_value(a+1,"adjusted_close")
    pricetwodaysago = StockHistory.get_value(a+2,"adjusted_close")
    pricethreedaysago = StockHistory.get_value(a+3,"adjusted_close")
    pricefourdaysago = StockHistory.get_value(a+4,"adjusted_close")
    pricefivedaysago = StockHistory.get_value(a+5,"adjusted_close")
    pricesixdaysago = StockHistory.get_value(a+6,"adjusted_close")
    pricesevendaysago = StockHistory.get_value(a+7,"adjusted_close")
    
    #Magnitude
    mag1 = (pricetoday-priceonedayago)/priceonedayago
    mag2 = (pricetoday-pricetwodaysago)/pricetwodaysago
    mag3 = (pricetoday-pricethreedaysago)/pricethreedaysago
    mag4 = (pricetoday-pricefourdaysago)/pricefourdaysago
    mag5 = (pricetoday-pricefivedaysago)/pricefivedaysago
    mag6 = (pricetoday-pricesixdaysago)/pricesixdaysago
    mag7 = (pricetoday-pricesevendaysago)/pricesevendaysago
    
    #Regression
    reg1 = getSlope(today, 5, 0) #5 Days in Past
    reg2 = getSlope(today, 10, 0) #10 Days in Past
    reg3 = getSlope(today, 15, 0) #15 Days in Past
    reg4 = getSlope(today, 20, 0) #20 Days in Past
    
    #Day Of The Week
    dayoftheweek = today.weekday() #0=Monday, 6=Sunday
    
    #Calculate Slope
    slope = getSlope(today, 5, 1)
    if (slope >= 0):
        slope = 1
    else:
        slope = 0
    
    #Log Data
    print("Article " + str(i) + " , Output: " + str(slope))
    with open('TrainingData.csv', 'a') as csvfile: #wb
        filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow([pos, neg, daysOld, mag1, mag2, mag3, mag4, mag5, mag6, mag7,
                             reg1, reg2, reg3, reg4, dayoftheweek, slope])

Collected data from NVDA for 4594 days!
Article 1 , Output: 0
Article 2 , Output: 0
Article 3 , Output: 0
Article 5 , Output: 0
Article 6 , Output: 1
Article 8 , Output: 0
Article 9 , Output: 1
Article 10 , Output: 0
Article 11 , Output: 1
Article 12 , Output: 1
Article 13 , Output: 1
Article 14 , Output: 1
Article 15 , Output: 1
Article 16 , Output: 1
Article 17 , Output: 1
Article 19 , Output: 1
Article 20 , Output: 1
Article 21 , Output: 1
Article 22 , Output: 1
Article 23 , Output: 1
Article 24 , Output: 1
Article 25 , Output: 0
Article 26 , Output: 1
Article 27 , Output: 1
Article 28 , Output: 1
Article 29 , Output: 1
Article 30 , Output: 1
Article 31 , Output: 1
Article 32 , Output: 1
Article 33 , Output: 1
Article 35 , Output: 1
Article 36 , Output: 0
Article 40 , Output: 1
Article 41 , Output: 0
Article 42 , Output: 0
Article 43 , Output: 0
Article 44 , Output: 1
Article 45 , Output: 1
Article 46 , Output: 0
Article 47 , Output: 1
Article 48 , Output: 0
Article 49 , Output: 0
A

#### Organize Data w/ 70% Split

In [218]:
df_raw = read_csv('TrainingDataYesterday.csv').sample(frac=1)
df = (df_raw - df_raw.mean()) / (df_raw.max() - df_raw.min()) #Normalization (rescaling from -1 to 1)
df.iloc[:,15] = df_raw.iloc[:,15].values #Replace Normalized Y with Original 0-1 Y

NumRows = df.shape[0]
split = 0.7
print("Number of Instances: " + str(NumRows))
X_train = df.iloc[0:int(NumRows*split),[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]].as_matrix()
X_test = df.iloc[int(NumRows*split):NumRows,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]].as_matrix()

Y_train = df.iloc[0:int(NumRows*split),[15]].as_matrix()
Y_test = df.iloc[int(NumRows*split):NumRows,[15]].as_matrix()

print("X Train: " , X_train.shape)
print("X Test : " , X_test.shape)
print("Y Train: " , Y_train.shape)
print("Y Test : " , Y_test.shape)

Number of Instances: 348
('X Train: ', (243, 15))
('X Test : ', (105, 15))
('Y Train: ', (243, 1))
('Y Test : ', (105, 1))


#### Create, Compile, & Fit Model

In [219]:
model = Sequential()
model.add(Dropout(0.2, input_shape=(15,)))
model.add(Dense(15, input_dim=15, init= 'uniform' , activation= 'relu' )) #4 4 3 1
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(30, init= 'uniform' , activation= 'relu' ))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(30, init= 'uniform' , activation= 'relu' ))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(4, init= 'uniform' , activation= 'relu' ))
model.add(BatchNormalization())
model.add(Dense(1, init= 'uniform' , activation= 'sigmoid' ))

model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])

model.fit(X_train, Y_train, validation_data=(X_test,Y_test), nb_epoch=100, batch_size=9)


model_json = model.to_json()
with open("SavedModel/new2model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("SavedModel/new2model.h5")
print("Saved model to disk")

/Users/ethanshaotran/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(15, activation="relu", kernel_initializer="uniform", input_dim=15)`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ethanshaotran/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="uniform")`
  
/Users/ethanshaotran/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="uniform")`
  if __name__ == '__main__':
/Users/ethanshaotran/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="relu", kernel_initializer="uniform")`
  if sys.path[0] == '':
/Users/ethanshaotran/anaconda/lib/python

Train on 243 samples, validate on 105 samples
Epoch 1/100
243/243 [==============================] - 0s - loss: 0.6927 - acc: 0.5021 - val_loss: 0.6908 - val_acc: 0.6571
Epoch 2/100
243/243 [==============================] - 0s - loss: 0.6851 - acc: 0.6049 - val_loss: 0.6903 - val_acc: 0.6571
Epoch 3/100
243/243 [==============================] - 0s - loss: 0.6808 - acc: 0.5802 - val_loss: 0.6882 - val_acc: 0.6571
Epoch 4/100
243/243 [==============================] - 0s - loss: 0.6712 - acc: 0.6214 - val_loss: 0.6868 - val_acc: 0.6571
Epoch 5/100
243/243 [==============================] - 0s - loss: 0.6623 - acc: 0.6543 - val_loss: 0.6847 - val_acc: 0.6571
Epoch 6/100
243/243 [==============================] - 0s - loss: 0.6557 - acc: 0.6543 - val_loss: 0.6817 - val_acc: 0.6571
Epoch 7/100
243/243 [==============================] - 0s - loss: 0.6290 - acc: 0.6872 - val_loss: 0.6787 - val_acc: 0.6571
Epoch 8/100
243/243 [==============================] - ETA: 0s - loss: 0.5774 - acc: 0

# Real-Life Usage

#### Ethan's Testing

In [43]:
    backorahead=1
    look_ahead=5
    articledate = '2018-03-07'
    
    t = SearchCSVForDate(articledate)
    
    #LEAST SQUARE METHOD - Find Line of Best Fit
    #Calculate X avg, Create list of X Values
    Xavg = 0
    Xlist = []
    if backorahead == 1:
        for x in range(t+1,t-look_ahead,-1): #MUST DO REVERSE FOR LOOP
            Xavg += x
            Xlist.append(x)
    else:
        for x in range(t+look_ahead,t,-1):
            Xavg += x
            Xlist.append(x)
    Xavg /= len(Xlist)
    Xlist.reverse()
        
    #Calculate Y avg, Create list of Y Values
    Yavg = 0
    Ylist = []
    if backorahead == 1:
        for x in range(t+1,t-look_ahead,-1):
            y = StockHistory.get_value(x,"adjusted_close")
            Yavg += y
            Ylist.append(y)
    else:
        for x in range(t+look_ahead,t,-1):
            y = StockHistory.get_value(x,"adjusted_close")
            Yavg += y
            Ylist.append(y)
    Yavg /= len(Ylist)
    
    print(Xlist)
    print(Ylist)
    
    TopCalculation = 0;
    for i in (0,len(Xlist)-1): #Top Part
        TopCalculation += (Xlist[i] - Xavg)*(Ylist[i] - Yavg)
        
    BottomCalculation = 0;
    for i in (0,len(Xlist)-1): #Top Part
        BottomCalculation += math.pow((Xlist[i] - Xavg), 2)
        
    Slope = TopCalculation/BottomCalculation
    print(Slope)

[8, 9, 10, 11, 12, 13]
[176.66999999999999, 175.03, 176.94, 179.97999999999999, 181.72, 179.97]
0.629615384615


In [232]:
LoadModel()
StockHistory = CollectData("AAPL")
Predict("AAPL", 0)

Loaded model from disk
Collected data from AAPL for 4594 days!
Found 25 Articles for Stock AAPL
22 instances found
[[ 0.90652955]
 [ 0.92966026]
 [ 0.98842245]
 [ 0.8955251 ]
 [ 0.96814555]
 [ 0.97546041]
 [ 0.86617571]
 [ 0.99619615]
 [ 0.68044579]
 [ 0.39871749]
 [ 0.9554168 ]
 [ 0.77140462]
 [ 0.9823525 ]
 [ 0.01395435]
 [ 0.99169534]
 [ 0.98842245]
 [ 0.64916801]
 [ 0.80858427]
 [ 0.99278939]
 [ 0.97958416]
 [ 0.91286802]]
21
AAPL is Predicted to RISE. +18,-2


array([ 0.79707855], dtype=float32)

#### [F] Prediction for One Stock - Main Block

In [21]:
def Predict(stocktopredict, verbosity):
    StockToAnalyze = stocktopredict
    Analyze(StockToAnalyze, verbosity)

    try: 
        df = read_csv("StockData/"+StockToAnalyze+'.csv')
    except IOError:
        print("Nothing found")
        return 0
        
    dfrows = df.shape[0]
    print(str(dfrows) + " instances found")
    if (dfrows > 1) and not (df.iloc[0,0] == 'error'):
        X_set = df.iloc[0:dfrows-1,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]].as_matrix() #CHANGED

        Y_pred = model.predict(X_set)
        print(Y_pred)

        Sum = 0
        PosArticles = 0
        NegArticles = 0
        for i in range(0,Y_pred.shape[0]-1):
            if (Y_pred[i] >= 0.5):
                PosArticles += 1
            else:
                NegArticles += 1
            Sum += Y_pred[i]
        print(len(Y_pred))
        finalstockresult = Sum/len(Y_pred)
                
        if (PosArticles > NegArticles):
            print(StockToAnalyze + " is Predicted to RISE. " + "+" + str(PosArticles) + ",-" + str(NegArticles))
        elif (PosArticles < NegArticles):
            print(StockToAnalyze + " is Predicted to FALL. " + "+" + str(PosArticles) + ",-" + str(NegArticles))
        elif (PosArticles == NegArticles):
            print(StockToAnalyze + " is Predicted to STAY. " + "+" + str(PosArticles) + ",-" + str(NegArticles))
        
        return finalstockresult
    else:
        print("Nothing found")
        return 0

#### [F] CSV Creation

In [37]:
def Analyze (stockname, verbosity):
    TickerName = stockname
    URLList = GetNewsURLs(stockname)

    if (len(URLList) > 0):
        #Clear the CSV if it currently exists:
        f = open("StockData/"+stockname+'.csv', "w+")
        f.close()
        
        #Write in columns headers:
        with open("StockData/"+stockname+'.csv', 'a') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
                filewriter.writerow(['Pos','Neg','DaysOld','Mag1','Mag2','Mag3','Mag4','Mag5','Mag6','Mag7','Reg1','Reg2','Reg3','Reg4','dayoftheweek'])
        
        for i in range(0,len(URLList)-1):
            url = URLList[i]
            if (verbosity == 1):
                print("Current URL: " + url)
            articletext, articledate = URLtoTextSummary(url)
            print(articletext)
            #Get Sentiment Value
            ListedText = articletext.split() #Turn into list
            pos, neg = getSentiment(ListedText)
            print("Pos, Neg: " + str(pos) + ", " + str(neg))
            
            if (articledate is None) or (isinstance(articledate, unicode)):
                continue
            
            # Days since Written
            today = datetime.now().replace(tzinfo=None)
            articledate = articledate.replace(tzinfo=None)
            strtoday = today.strftime('%Y-%m-%d')
            while BusinessDay(strtoday) == False:
                today = today - timedelta(days=1)
                strtoday = today.strftime('%Y-%m-%d')
            daysOld = (today - articledate).days
    
            #Price "today"
            a = SearchCSVForDate(strtoday)
            pricetoday = StockHistory.get_value(a,"adjusted_close")
            priceonedayago = StockHistory.get_value(a+1,"adjusted_close")
            pricetwodaysago = StockHistory.get_value(a+2,"adjusted_close")
            pricethreedaysago = StockHistory.get_value(a+3,"adjusted_close")
            pricefourdaysago = StockHistory.get_value(a+4,"adjusted_close")
            pricefivedaysago = StockHistory.get_value(a+5,"adjusted_close")
            pricesixdaysago = StockHistory.get_value(a+6,"adjusted_close")
            pricesevendaysago = StockHistory.get_value(a+7,"adjusted_close")
    
            #Magnitude
            mag1 = (pricetoday-priceonedayago)/priceonedayago
            mag2 = (pricetoday-pricetwodaysago)/pricetwodaysago
            mag3 = (pricetoday-pricethreedaysago)/pricethreedaysago
            mag4 = (pricetoday-pricefourdaysago)/pricefourdaysago
            mag5 = (pricetoday-pricefivedaysago)/pricefivedaysago
            mag6 = (pricetoday-pricesixdaysago)/pricesixdaysago
            mag7 = (pricetoday-pricesevendaysago)/pricesevendaysago
    
            #Regression
            reg1 = getSlope(today, 5, 0) #Past
            reg2 = getSlope(today, 10, 0) #Past
            reg3 = getSlope(today, 15, 0) #Past
            reg4 = getSlope(today, 20, 0) #Past
    
            #DayOfTheWeek
            dayoftheweek = today.weekday() #0=Monday, 6=Sunday - should only be 0-4
    
            #Log to CSV:
            with open("StockData/"+stockname+'.csv', 'a') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
                filewriter.writerow([pos, neg, daysOld, mag1, mag2, mag3, mag4, mag5, mag6, mag7,
                             reg1, reg2, reg3, reg4, dayoftheweek])
            #END FOR
        
        
    else: #Gives CSV with Error Message
        with open("StockData/"+stockname+'.csv', 'a') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
                filewriter.writerow(['error'])

In [18]:
def getCurrentPrice (stockticker):
    import pandas as pd
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=" + stockticker + "&interval=1min&apikey=" + "VPRQOQ85W9KS1DXH"+"&datatype=csv" #apikey 
    StockData = pd.read_csv(url)
    return StockData.iloc[0,4]

In [19]:
def LoadModel():
    json_file = open('SavedModel/new2model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights("SavedModel/new2model.h5")
    print("Loaded model from disk")
    return loaded_model

In [ ]:
#Add words to Negative Dictionary:
Dictionary = open('NegativeWords.txt', 'r+')  
MD = pd.read_csv('NegWordsMD.csv')  
RealMD = pd.read_csv('MasterDictionary.csv')  
   
for i in range(0,RealMD.shape[0]):  
    if (RealMD.get_value(i,"Negative") > 0):  
        word = RealMD.get_value(i,"Word")  
        Dictionary.write("_"+word+"_\n")  
          
for i in range(0,MD.shape[0]):  
    if not ("_"+MD.iloc[i][0]+"_" in Dictionary):  
        word = MD.iloc[i][0]  
        Dictionary.write("_"+word+"_\n")  
Dictionary.close()    

#Add words to Positive Dictionary:  
Dictionary = open('PositiveWords.txt', 'r+')  
MD = pd.read_csv('PosWordsMD.csv')  
RealMD = pd.read_csv('MasterDictionary.csv')   
  
for i in range(0,RealMD.shape[0]):  
    if (RealMD.get_value(I,"Positive") > 0):  
        word = RealMD.get_value(i,"Word")  
        Dictionary.write("_"+word+"_\n")  
          
for i in range(0,MD.shape[0]):  
    if not ("_"+MD.iloc[i][0]+"_" in Dictionary):  
        word = MD.iloc[i][0]  
        Dictionary.write("_"+word+"_\n")  
Dictionary.close()  


In [ ]:
model = LoadModel()
StockHistory = CollectData("SNAP")
Predict("SNAP", 1)

Loaded model from disk
Collected data from SNAP for 307 days!
Found 27 Articles for Stock SNAP
Current URL: http://news.google.com/news/url?sa=t&fd=R&ct2=us&usg=AFQjCNEZoI9dwN4Zm_qZshehGE539o4beg&clid=c3a7d30bb8a4878e06b80cf16b898331&ei=egkAW9u-J4z4hgHY35uYAQ&url=https://www.investopedia.com/news/snaps-stock-seen-plunging-16-new-record-low/
This test image from one of the four cameras aboard the Transiting Exoplanet Survey Satellite (TESS) captures a swath of the southern sky along the plane of our galaxy.
TESS is expected to cover more than 400 times the amount of sky shown in this image when using all four of its cameras during science operations.
More than 200,000 stars are visible in this image.
The image, which is centered in the constellation Centaurus, includes dark tendrils from the Coalsack Nebula and the bright emission nebula Ced 122 (upper right).The bright star at bottom center is Beta Centauri.
Credit: NASA/MIT/TESS
Pos, Neg: 1, 1
Current URL: http://news.google.com/news/

This test image from one of the four cameras aboard the Transiting Exoplanet Survey Satellite (TESS) captures a swath of the southern sky along the plane of our galaxy.
TESS is expected to cover more than 400 times the amount of sky shown in this image when using all four of its cameras during science operations.
More than 200,000 stars are visible in this image.
The image, which is centered in the constellation Centaurus, includes dark tendrils from the Coalsack Nebula and the bright emission nebula Ced 122 (upper right).The bright star at bottom center is Beta Centauri.
Credit: NASA/MIT/TESS
Pos, Neg: 1, 1
Current URL: http://news.google.com/news/url?sa=t&fd=R&ct2=us&usg=AFQjCNFfiCRGcRwBNtfORqPDNSrqObPUWQ&clid=c3a7d30bb8a4878e06b80cf16b898331&cid=52779893077714&ei=egkAW9u-J4z4hgHY35uYAQ&url=https://www.thestreet.com/investing/stocks/snap-cfo-departure-could-mark-turning-point-14583774
This test image from one of the four cameras aboard the Transiting Exoplanet Survey Satellite (TESS)

People who search for aliens on Earth are likely thinking of little green men and flying saucers in the sky, but what if they should be looking at tentacles in the ...
Could Life on Earth Have Come From Space?
Pos, Neg: 0, 0
Current URL: http://news.google.com/news/url?sa=t&fd=R&ct2=us&usg=AFQjCNF6EBBudbxRIfanDVTRMC1s27FlLQ&clid=c3a7d30bb8a4878e06b80cf16b898331&ei=egkAW9u-J4z4hgHY35uYAQ&url=https://finance.yahoo.com/news/trade-snap-inc-150-return-144216342.html
